In [44]:
import pandas as pd

df = pd.read_csv("new.csv")

print(df.columns)

Index(['Unnamed: 0', 'date', 'weather_code', 'temperature_2m_mean',
       'temperature_2m_max', 'temperature_2m_min', 'wind_speed_10m_max',
       'wind_gusts_10m_max', 'wind_direction_10m_dominant',
       'shortwave_radiation_sum', 'et0_fao_evapotranspiration', 'location'],
      dtype='object')


C:\Users\PC\AppData\Local\Temp\ipykernel_23788\3087352913.py:3: DtypeWarning: Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("new.csv")


In [47]:
# Create next day weather code feature by location
df['next_day_weather_code'] = df['weather_code'].shift(-1)


In [48]:
import numpy as np
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report

import matplotlib.pyplot as plt

# Drop rows with NaN values in critical columns
df_clean = df.dropna(subset=['weather_code', 'next_day_weather_code'])

# # Convert date to datetime if it's not already
# df_clean['date'] = pd.to_datetime(df_clean['date'])

# # Extract time-based features
# df_clean['year'] = df_clean['date'].dt.year
# df_clean['month'] = df_clean['date'].dt.month
# df_clean['day'] = df_clean['date'].dt.day
# df_clean['dayofweek'] = df_clean['date'].dt.dayofweek

# One-hot encode location
# df_clean = pd.get_dummies(df_clean, columns=['location'])

# Define features and target
X = df_clean.drop(['next_day_weather_code', 'date', 'Unnamed: 0','location'], axis=1)
y = df_clean['next_day_weather_code']


# Use LabelEncoder to transform weather codes to consecutive integers
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# Print original classes and their encoded values
# print("Original weather codes and their encoded values:")
# for i, orig_class in enumerate(label_encoder.classes_):
#     print(f"Original: {orig_class} -> Encoded: {i}")

# Split data using encoded labels
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

# Train XGBoost classifier
model = xgb.XGBClassifier(max_depth=5, learning_rate=0.1, n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Make predictions
y_pred = model.predict(X_test)

# Convert predictions back to original weather codes for interpretation
y_pred_original = label_encoder.inverse_transform(y_pred)
y_test_original = label_encoder.inverse_transform(y_test)

# # Evaluate the model
print(f"Accuracy: {accuracy_score(y_test, y_pred)*100:.4f}")
# print("\nClassification Report:")
# print(classification_report(y_test, y_pred, target_names=[str(c) for c in label_encoder.classes_]))

# Print a sample of predictions (original weather codes)
print("\nSample predictions (original weather codes):")
sample_indices = range(min(10, len(y_test)))
for i in sample_indices:
    print(f"True: {y_test_original[i]}, Predicted: {y_pred_original[i]}")
# Feature importance analysis
print(classification_report(y_test, y_pred))

# Feature importance
importance = model.feature_importances_
feature_names = X.columns
feature_importance = pd.DataFrame({'Feature': feature_names, 'Importance': importance})
feature_importance = feature_importance.sort_values('Importance', ascending=False)
print("\nTop 10 Important Features:")
print(feature_importance.head(10))

# Visualize feature importance
# plt.figure(figsize=(12, 6))
# plt.barh(feature_importance['Feature'][:10], feature_importance['Importance'][:10])
# plt.xlabel('Importance')
# plt.ylabel('Feature')
# plt.title('Top 10 Feature Importance for Next Day Weather Code Prediction')
# plt.tight_layout()
# plt.show()

Accuracy: 50.9360

Sample predictions (original weather codes):
True: 0.0, Predicted: 0.0
True: 2.0, Predicted: 0.0
True: 1.0, Predicted: 3.0
True: 3.0, Predicted: 3.0
True: 0.0, Predicted: 3.0
True: 0.0, Predicted: 0.0
True: 2.0, Predicted: 0.0
True: 3.0, Predicted: 3.0
True: 0.0, Predicted: 0.0
True: 3.0, Predicted: 3.0
              precision    recall  f1-score   support

           0       0.59      0.76      0.66      4466
           1       0.31      0.02      0.04      1781
           2       0.33      0.00      0.01      1745
           3       0.49      0.83      0.62      7006
           4       0.26      0.14      0.18      1318
           5       0.33      0.00      0.00       482
           6       0.00      0.00      0.00       152
           7       0.24      0.04      0.07       284
           8       0.00      0.00      0.00       118
           9       0.00      0.00      0.00         4
          10       0.31      0.10      0.16       864
          11       0.48    

d:\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
d:\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
d:\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
